# Muhamad Fikri Nurohman / 18220097
## Tugas 1 Calon Asisten Basis Data

Deskripsi data : Data yang diambil adalah data produk TWS (True Wireless Stereo) dari situs amazon.com.
Beberapa informasi yang diambil terkait produk adalah nama, harga semula, harga setelah diskon, persentase diskon, rating, jumlah review, dan url produk.

1. Import library yang diperlukan

In [1]:
import csv
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

2. Membuat fungsi get_url.
Fungsi ini menerima argumen search_term yang merupakan kata kunci yang akan digunakan untuk men-generate URL. URL tersebut kemudian akan dipakai untuk mengambil data

In [2]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&i=electronics&rh=n%3A172541&qid=1656312774&ref=sr_pg_1'
    
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page={}'
    
    return url

3. Membuat prosedur extract_record.
Prosedur ini menerima argumen berupa item yang akan diambil data-data yang diperlukannya. Item tersebut masih berupa html tag dan diproses sedemikian sehingga data-data yang diperlukan terkait produk TWS dari situs amazon.com dapat disimpan dalam variabel-variabel tertentu. Kumpulan data tersebut kemudian dijadikan sebuah array of data yang diperlukan (dalam hal ini variabel result).

In [3]:
def extract_record(item):
    
    #1. Mengambil data nama dan url produk
    atag = item.h2.a
    name = atag.text.strip()
    url = 'https://www.amazon.con'+atag.get('href')
    
    #2. Mengambil data persentase diskon produk
    try:
        discount_percentage = item.find('span', {'class': 'a-size-extra-large s-color-discount s-light-weight-text'}).text
        discount_processed = discount_percentage[1:]
        discount = float(discount_processed[:-1])
    except AttributeError:
        discount_percentage = '0%'
        discount = 0
    
    #3. Mengambil data harga setelah diskon produk
    try:
        price_parent = item.find('span', 'a-price')
        dcp = price_parent.find('span', 'a-offscreen').text
        discount_price = float(dcp[1:])     
    except AttributeError:
        return
        
    #4. Mengambil data harga sebelum diskon produk
    if discount == 0:
        normal_price = discount_price
    else:
        try:
            normal_price_parent = item.find('span', {'class': 'a-price a-text-price'})
            np = normal_price_parent.find('span', 'a-offscreen').text
            normal_price = float(np[1:])
        except AttributeError:
            pass
    
    #5. Mengambil data rating dan jumlah review produk
    try:
        rating = item.i.text
        review = item.find('span', {'class': 'a-size-base s-underline-text'}).text
        review_processed = review.replace(",", "")
        review_count = int(review_processed)
    except AttributeError:
        rating = ''
        review_count = 0
        
    if rating != "":
        rating_clear = rating.split()[0]
        rating_in_float = float(rating_clear)
    else:
        rating_in_float = 0
    
    result = (name, normal_price, discount_price, discount, rating_in_float, review_count, url)
    
    return result

5. Membuat prosedur csv_to_json.
Prosedur ini digunakan untuk mengubah data-data pada sebuah file csv menjadi bentuk Javascript Object Notation (JSON). Hal ini diperlukan karena hasil scraping pada kode yang saya buat adalah berbentuk csv. Jadi, prosedur ini diperlukan untuk mendapatkan hasil dalam bentuk json.

In [4]:
def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #Menulis file csv
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #Me-load file csv menggunakan library dictionary reader
        csvReader = csv.DictReader(csvf) 

        #Mengubah setiap baris csv menjadi dictionary python
        for row in csvReader: 
            #Menambah python dictionary ini ke array jsonArray
            jsonArray.append(row)
  
    #Mengubah jsonArray menjadi JSON String dan menulis ke dalam file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)

6. Membuat fungsi main.
Fungsi ini dapat dikatakan sebagai program utama pada rangkaian kode pada file ini. Fungsi ini menerima argumen search_term yang merupakan kata kunci yang akan digunakan untuk men-generate URL. Hasil dari fungsi main ini adalah berupa file csv dan json berisi data-data TWS yang diambil dari situs amazon.com

In [5]:
def main(search_term):
    #Menginstall webdriver chrome untuk melakukan scraping data
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    records = []
    url = get_url(search_term)
    
    #Dalam hal ini, saya hanya melakukan scraping pada 30 page awal produk TWS di situs amazon.com
    for page in range(1, 30):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        #Mengekstrak data pada setiap item setelah dilakukan pencarian
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    
    #Driver harus ditutup setelah digunakan
    driver.close()
    
    #Menulis csv file
    with open('TWS_amazon.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Normal Price (USD)', 'Discounted Price (USD)', 'Discount percentage (%)', 'Rating (out of 5 stars)', 'Review Count', 'Url'])
        writer.writerows(records)
    
    #Nama file setelah melakukan scraping
    csvFilePath = r'TWS_amazon.csv'
    jsonFilePath = r'TWS_amazon.json'
    
    #Konversi csv menjadi file json
    csv_to_json(csvFilePath, jsonFilePath)
    
    #Pesan di akhir proses scraping. Juga sebagai indikator banyaknya jumlah data yang diambil
    print("Proses scraping berhasil")
    print("Berhasil melakukan scraping sebanyak {} data".format(len(records)))

7. Melakukan pemanggilan fungsi main dengan argumen 'tws'

In [6]:
main('tws')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Uning Rohani\.wdm\drivers\chromedriver\win32\102.0.5005.61]


Proses scraping berhasil
Berhasil melakukan scraping sebanyak 819 data
